In [ ]:
# %pip install llama-index-llms-ollama
# %pip install llama-index-ollama-embeddings
# %pip install llama-index llama-index-vector-stores-lancedb

In [22]:
import os
import tqdm
import json
from typing import List, Dict, Optional
import logging

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings,
    Document
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool, ToolOutput
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

from llama_index.vector_stores.lancedb import LanceDBVectorStore


### Setup LLM and Embedding Model in Settings Object

In [2]:
llm = Ollama(model="llama3.1", base_url="http://localhost:11434")
embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

Settings.chunk_size = 512
Settings.embed_model = embed_model

### Vector Stores Setup

In [3]:
problems_vector_store = LanceDBVectorStore(
    uri="./lancedb", 
    table_name="problems_table",
    mode="overwrite"
)

parts_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="parts_table",
    mode="overwrite",
)

diagnostics_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="diagnostics_table",
    mode="overwrite",
)

cost_estimates_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="cost_estimates_table",
    mode="overwrite",
)

maintenance_schedules_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="maintenance_schedules_table",
    mode="overwrite",
)

cars_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="car_maintenance_table",
    mode="overwrite",
)

In [4]:
def load_and_index_document_from_file(
        file_path: str,
        vector_store: LanceDBVectorStore
) -> VectorStoreIndex:
    """load and index a document from a file"""
    with open(file_path, "r") as file:
        data = json.load(file)
        document = Document(text=json.dumps(data))
    parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
    nodes = parser.get_nodes_from_documents([document])
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    return VectorStoreIndex(nodes, storage_context=storage_context)

In [5]:
def create_retriever(index: VectorStoreIndex) -> VectorIndexRetriever:
    return index.as_retriever(similarity_top_k=6)

Load and index documents directly from file paths

In [6]:
problems_index = load_and_index_document_from_file(
    file_path="./data/problems.json",
    vector_store=problems_vector_store,
)

parts_index = load_and_index_document_from_file(
    file_path="./data/parts.json",
    vector_store=parts_vector_store
)

cars_index = load_and_index_document_from_file(
    file_path="./data/cars_models.json",
    vector_store=cars_vector_store
)

cost_estimates_index = load_and_index_document_from_file(
    file_path="./data/cost_estimates.json",
    vector_store=cost_estimates_vector_store
)

diagnostics_index = load_and_index_document_from_file(
    file_path="./data/diagnostics.json",
    vector_store=diagnostics_vector_store
)

maintenance_schedules_index = load_and_index_document_from_file(
    file_path="./data/maintenance.json",
    vector_store=maintenance_schedules_vector_store
)

Create Retrievers

In [7]:
problems_retriever = create_retriever(problems_index)
parts_retriever = create_retriever(parts_index)
cars_retriever = create_retriever(cars_index)
cost_estimates_retriever = create_retriever(cost_estimates_index)
diagnostics_retriever = create_retriever(diagnostics_index)
maintenance_schedules_retriever = create_retriever(maintenance_schedules_index)

Testing the retrievers and query engine

In [8]:
sample_query = "My brake pad isn't working or I don't know, but the brakes are poor, and by the way, what's the cost for the solution?"
sample_query_engine = cost_estimates_index.as_query_engine(llm=llm, embed_model=embed_model)
sample_retrieved_nodes = cost_estimates_retriever.retrieve(sample_query)

sample_response = sample_query_engine.query(sample_query)

print("Response: ", sample_response)

# Print only relevant information from results
for result in sample_retrieved_nodes:
    print(f"Result - Node ID: {result.node_id}")
    print(f"Relevant Text: {result.text[:150]}...")
    print(f"Score: {result.score:.3f}")

Response:  A brake pad replacement is likely needed. The average cost for this repair is $150, with a minimum of $100 and a maximum of $300.
Result - Node ID: a9dffbb2-7565-4f37-a063-a232b460133c
Relevant Text: "max": 600}}, {"repair": "Fuel Pump Replacement", "average_cost": 500, "cost_range": {"min": 400, "max": 700}}, {"repair": "AC Compressor Replacement"...
Score: 0.000
Result - Node ID: f799fd76-8ee9-4bbf-8975-2886b58ec85c
Relevant Text: [{"repair": "Brake pad replacement", "average_cost": 150, "cost_range": {"min": 100, "max": 300}}, {"repair": "Oil change", "average_cost": 50, "cost_...
Score: 0.000


#### Creating Functools and setting up the agent

In [9]:
max_context_information = 200

1. Retrieval Tools

In [18]:
def retrieve_problems(query: str) -> str:
    """Searches the problem catalog to find relevant automotive problems for given query"""
    docs = problems_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information

def retrieve_parts(query: str) -> str:
    """Searches the parts catalog to find relevant automotive parts for given query"""
    docs = parts_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information

def retrieve_cars(query: str) -> str:
    """Searches the cars catalog to find relevant automotive cars for given query"""
    docs = cars_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information

def retrieve_cost_estimates(query: str) -> str:
    """Searches the cost estimates catalog to find relevant automotive cost estimates for given query"""
    docs = cost_estimates_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information

def retrieve_diagnosis(query: str) -> str:
    """Searches the diagnostics catalog to find relevant automotive diagnostics for given query"""
    docs = diagnostics_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information

def retrieve_maintenance_schedules(query: str) -> str:
    """Searches the maintenance schedules catalog to find relevant automotive maintenance schedules for given query"""
    docs = maintenance_schedules_retriever.retrieve(query)
    information = str([doc.text[:max_context_information] for doc in docs])
    return information    

2. Additional Tools

In [19]:
def comprehensive_diagnosis(symptoms: str) -> str:
    """
    Provides a comprehensive diagnostic including possible causes, estimated costs, and required parts.

    Args:
        symptoms: A string describing the symptoms of the car
    Returns:
        A string with the comprehensive diagnostic report
    """
    #use existing tools to get information
    possible_causes = retrieve_diagnosis(symptoms)

    #extract the most likely cause (this is a simplification)
    likely_cause = possible_causes[0] if possible_causes else "Unknown issue"
    estimated_cost = retrieve_cost_estimates(likely_cause)
    required_parts = retrieve_parts(likely_cause)

    #construct the report
    report = f"Comprehensive Diagnostic Report\n\n"
    report += f"Symptoms: {symptoms}\n\n"
    report += f"Possible Causes: \n{possible_causes}\n\n"
    report += f"Most Likely Cause: \n{likely_cause}\n\n"
    report += f"Estimated Cost: \n{estimated_cost}\n\n"
    report += f"Required Parts: \n{required_parts}\n\n"
    report += "Please note that this is a simplified diagnostic report. For a detailed and accurate diagnosis, consider consulting a professional mechanic."
    
    return report

In [12]:
def get_car_model_info(mileage: int, car_make: str, car_model: str, car_year: int) -> dict:
    """Retrieve car model information from cars_models.json file"""
    with open("./data/cars_models.json", "r") as file:
        cars_models = json.load(file)

        for car in cars_models:
            if (
                car["car_make"].lower() == car_make.lower() and
                car["car_model"].lower() == car_model.lower() and
                car["car_year"] == car_year
            ):
                return car
    return {}    
    

In [13]:
def retrieve_car_details(make:str, model:str, year:int) -> str:
    """Retrieves the make, model, and year of the car and return the common issues if any"""
    car_details = get_car_model_info(0, make, model, year)
    if car_details:
        return f"{year} {make} {model} might have the following common issues: {', '.join(car_details['common_issues'])}"
    return f"No common issues found for {make} {model} {year}."



In [15]:
def plan_maintenance(mileage:int, car_make:str, car_model:str, car_year:int) -> str:
    """
    Creates a comprehensive maintenance plan based on the car's mileage and details.

    Args:
        mileage: The current mileage of the car.
        car_make: The make of the car.
        car_model: The model of the car.
        car_year: The year the car was manufactured.

    Returns:
        A string with a comprehensive maintenance plan.
    """
    car_details = retrieve_car_details(car_make, car_model, car_year)
    car_model_info = get_car_model_info(0, car_make, car_model, car_year)
    
    plan = f"Maintenance Plan for {car_make} {car_model} {car_year} at {mileage} miles\n\n"
    plan += f"Car Details: {car_details}\n\n"

    if car_model_info:
        plan += f"Common Issues:\n"
        for issue in car_model_info['common_issues']:
            plan += f"- {issue}\n"
        plan += f"\nEstimated Time: {car_model_info['estimated_time']}\n\n"
    else:
        plan += f"No common issues found for {car_make} {car_model} {car_year}."
    
    


In [17]:
from datetime import datetime, timedelta

def create_calander_invite(event_type: str, car_details: str, duration: int = 60) -> str:
    """
    Simulates creating a calendar invite for a car maintenance or repair event.

    Args:
        event_type: The type of event (e.g., "Oil Change", "Brake Inspection").
        car_details: Details of the car (make, model, year).
        duration: Duration of the event in minutes (default is 60).

    Returns:
        A string describing the calendar invite.
    """
    event_date = datetime.now() + timedelta(days=7)
    event_time = event_date.replace(hour=10, minute=0, second=0, microsecond=0)

    invite = f"Calendar Invite Created:\n\n"
    invite += f"Event: {event_type} for {car_details}\n"
    invite += f"Date: {event_time.strftime('%Y-%m-%d')}\n"
    invite += f"Time: {event_time.strftime('%I:%M %p')}\n"
    invite += f"Duration: {duration} minutes\n"
    invite += f"Location: Your Trusted Auto Shop, 90 Main St, Toronto, Canada\n\n"

    return invite

In [20]:
def coordinate_car_care(query: str, car_make: str, car_model: str, car_year: int, mileage: int) -> str:
    """
    Coordinates overall car care by integrating diagnosis, maintenance planning, and scheduling.

    Args:
        query: The user's query or description of the issue.
        car_make: The make of the car.
        car_model: The model of the car.
        car_year: The year the car was manufactured.
        mileage: The current mileage of the car.

    Returns:
        A string with a comprehensive car care plan.
    """
    car_details = retrieve_car_details(car_make, car_model, car_year)
    if "problem" in query.lower() or "issue" in query.lower():
        diagnosis = comprehensive_diagnosis(query)
        plan = f"Based on the query, here is a diagnosis: {diagnosis}\n\n"
        likely_cause = diagnosis.split("Most Likely Cause: \n")[1].split("\n")[0].strip()
        invite = create_calander_invite("Car Repair: {likely_cause}", car_details)
        plan += f"I've prepared a calendar invite for the repair:\n\n{invite}\n\n"
    else:
        maintenance_plan = plan_maintenance(mileage, car_make, car_model, car_year)
        plan += f"Based on the query, here is a maintenance plan: {maintenance_plan}\n\n"
        next_task = maintenance_plan.split("Task: ")[1].split("\n")[0].strip()
        invite = create_calander_invite(f"Maintenance: {next_task}", car_details)
        plan += f"I've prepared a calendar invite for your next maintenance task:\n\n{invite}\n\n"
    
    plan += "Remember to consult with a professional mechanic for presonalized advice and services."

    return plan    

In [21]:
retrieve_problems_tool = FunctionTool.from_defaults(fn=retrieve_problems)
retrieve_parts_tool = FunctionTool.from_defaults(fn=retrieve_parts)
diagnostic_tool = FunctionTool.from_defaults(fn=retrieve_diagnosis)
cost_estimates_tool = FunctionTool.from_defaults(fn=retrieve_cost_estimates)
maintenance_schedules_tool = FunctionTool.from_defaults(fn=retrieve_maintenance_schedules)
comprehensive_diagnosis_tool = FunctionTool.from_defaults(fn=comprehensive_diagnosis)
maintenance_plan_tool = FunctionTool.from_defaults(fn=plan_maintenance)
calender_invite_tool = FunctionTool.from_defaults(fn=create_calander_invite)
car_care_coordinator_tool = FunctionTool.from_defaults(fn=coordinate_car_care)
retrieve_car_details_tool = FunctionTool.from_defaults(fn=retrieve_car_details)

tools = [
    retrieve_problems_tool,
    retrieve_parts_tool,
    diagnostic_tool,
    cost_estimates_tool,
    maintenance_schedules_tool,
    comprehensive_diagnosis_tool,
    maintenance_plan_tool,
    calender_invite_tool,
    car_care_coordinator_tool,
    retrieve_car_details_tool
]


In [23]:
def reset_agent_memory():
    global agent_worker, agent
    agent_worker = FunctionCallingAgentWorker.from_tools(tools, llm=llm, verbose=True)
    agent = AgentRunner(agent_worker)

#intialize the agent
reset_agent_memory()

In [24]:
response = agent.chat("My car has 20,000 miles on it and I need to know when to change the oil?")

Added user message to memory: My car has 20,000 miles on it and I need to know when to change the oil?
=== LLM Response ===
Typically, oil changes are recommended every 5,000 to 7,500 miles for most vehicles. However, this can vary depending on the type of vehicle you have.

Here are some general guidelines:

*   For gasoline engines: Change oil every 5,000 to 7,500 miles.
*   For diesel engines: Change oil every 10,000 to 15,000 miles.
*   If your car has a turbocharger: Change oil every 3,000 to 5,000 miles.
*   If you live in an area with extreme temperatures (very hot or very cold): Change oil every 3,000 to 5,000 miles.

It's always best to check your owner's manual for the recommended oil change interval for your specific vehicle.


In [25]:
agent.chat("What will be the cost of an oil change ?")

Added user message to memory: What will be the cost of an oil change ?
=== LLM Response ===
The cost of an oil change can vary depending on several factors, including:

*   The type of oil used (conventional, synthetic, or high-performance)
*   The location where you get the oil changed ( dealership, independent mechanic, or quick lube)
*   Any additional services performed during the oil change (such as a tire rotation or brake pad inspection)

On average, here are some estimated costs for an oil change:

*   Conventional oil change: $25 to $50
*   Synthetic oil change: $40 to $70
*   High-performance oil change: $60 to $90

It's also worth noting that some oil changes may include additional services, such as a tire rotation or brake pad inspection, which can add to the overall cost. Always check with your mechanic for an accurate estimate of the cost.

In general, it's recommended to get an oil change every 5,000 to 7,500 miles to keep your engine running smoothly and prolong its lif

AgentChatResponse(response="The cost of an oil change can vary depending on several factors, including:\n\n*   The type of oil used (conventional, synthetic, or high-performance)\n*   The location where you get the oil changed ( dealership, independent mechanic, or quick lube)\n*   Any additional services performed during the oil change (such as a tire rotation or brake pad inspection)\n\nOn average, here are some estimated costs for an oil change:\n\n*   Conventional oil change: $25 to $50\n*   Synthetic oil change: $40 to $70\n*   High-performance oil change: $60 to $90\n\nIt's also worth noting that some oil changes may include additional services, such as a tire rotation or brake pad inspection, which can add to the overall cost. Always check with your mechanic for an accurate estimate of the cost.\n\nIn general, it's recommended to get an oil change every 5,000 to 7,500 miles to keep your engine running smoothly and prolong its lifespan.", sources=[], source_nodes=[], is_dummy_str

In [26]:
response = agent.chat(
    "I have a honda accor of 2017 model and it's mileage is 30000 right now, what are some common issues?"
)

Added user message to memory: I have a honda accor of 2017 model and it's mileage is 30000 right now, what are some common issues?
=== LLM Response ===
A 2017 Honda Accord with 30,000 miles is still relatively young and has likely had regular maintenance done on it. However, like any vehicle, it may start to experience some common issues as it ages.

Here are some potential problems to be aware of:

*   **Faulty Oxygen Sensor:** The oxygen sensor is a crucial component that monitors the air-fuel mixture in your engine. A faulty oxygen sensor can cause the engine to run rich or lean, which can lead to decreased performance and fuel efficiency.
*   **Worn Out Timing Chain:** The timing chain is responsible for keeping the engine's valves in sync with its pistons. Over time, it may wear out, causing problems like rough idling, stalling, or even engine failure.
*   **Faulty CV Boots:** The CV (Constant Velocity) boots connect your car's axles to its wheels. If they crack or tear, it can ca

In [27]:
response = agent.chat(
    "Can you help me with these issues, I want to do some maintenance and what's the cost for all of this services? for parts and all which will be required"
)

Added user message to memory: Can you help me with these issues, I want to do some maintenance and what's the cost for all of this services? for parts and all which will be required
=== LLM Response ===
I can provide information on the potential costs associated with the services you mentioned.

**Costs:**

*   **Faulty Oxygen Sensor:** The cost of a new oxygen sensor varies depending on the type and quality. On average, it can range from $20 to $50.
*   **Worn Out Timing Chain:** A timing chain replacement typically involves replacing both the timing chain and the water pump. This can cost between $500 to $1,000.
*   **Faulty CV Boots:** The cost of a new CV boot set varies depending on the quality and brand. On average, it can range from $20 to $50 for a single boot or $100 to $200 for a complete set.
*   **Loose or Worn Out Belt Tensioners:** A new belt tensioner typically costs between $10 to $30.
*   **Clogged Air Filter:** The cost of a new air filter varies depending on the type